In [2]:
import numpy as np
import netCDF4 as cdf
import pandas as pd
import xesmf as xe
import xarray as xr
import warnings

In [3]:
domains        = 1                                                                    # check this always
w              = '../../../DATA/WRF/wrfinput/'   # check this always
w_geo          = '../../../DATA/WRF/domains/4_domains/'# check this always
pre            = 'wrfinput_d0'
pre2           = 'geo_em.d0'
sim_time       = '2022-08-01 00:00:00','2022-08-31 23:00:00' 
dates          = pd.to_datetime(sim_time[0]).strftime('%Y-%m-%d')
pos            = '.nc'  
projection_wrf = 'Lambert Conformal'

In [4]:
dom              = domains
wrf_inp          = w + pre  + str(domains)
wrf_geo_p        = w_geo + pre2 + str(domains) + pos

In [5]:
#### 
edgar_path       = '../../../DATA/sources/ANTHRO/global/'
var              = ['CO','CO2','CH4'] # remember to include the other options
mvar             = [28.01,44.01,16.043]
lat              =  'lat'
lon              = 'lon'
edgar_out        = '../../../DATA/sources/ANTHRO/regrid/'
regrid_method    = 'conservative'

In [6]:
print('STARTING WITH GHG fire emissions REGRID-PREPROCESSING')
wrf_i          = cdf.Dataset(wrf_inp)
wrf_geo        = cdf.Dataset(wrf_geo_p)
wrf_lat        = np.array(wrf_geo.variables['XLAT_M'])
wrf_lon        = np.array(wrf_geo.variables['XLONG_M'])
wrf_lmask      = np.array(wrf_geo.variables['LANDMASK'])
wrf_lat_c      = np.array(wrf_geo.variables['XLAT_C'])
wrf_lon_c      = np.array(wrf_geo.variables['XLONG_C'])

STARTING WITH GHG fire emissions REGRID-PREPROCESSING


In [10]:
edgar_data = {}; edgar_units = {}
for v,pol in enumerate(var):
    edgar_open            = xr.open_dataset(edgar_path+'v6.0_'+var[v]+'_TOTAL_NO_FIRES_2018.0.1x0.1.nc')
    edgar_open['lon_adj'] = xr.where(edgar_open[lon]> 180, edgar_open[lon] - 360, edgar_open[lon])
    edgar_open            = edgar_open.sortby(edgar_open["lon_adj"])
    
    edgar_data[str(pol)]  = np.array(edgar_open['emis_tot'][7,:,:])*36*(10**11)/float(mvar[v]) 
    edgar_units[str(pol)] = edgar_open['emis_tot'].units
    
    lons_edgar     = np.array(edgar_open['lon_adj'])
    lats_edgar     = np.array(edgar_open[lat])

In [12]:
### aqui las longitudes ya son un vector.
lon_res        = abs(lons_edgar[1]-lons_edgar[0])
lat_res        = abs(lats_edgar[1]-lats_edgar[0])

srclon_cen     = lons_edgar
srclat_cen     = lats_edgar

#### estas lineas fueron actualizadas.
srclat_cor  = np.linspace(lats_edgar[0]+lat_res/2,lats_edgar[-1]-lat_res/2,lats_edgar.shape[0]+1)
srclon_cor  = np.arange(lons_edgar[0]-lon_res/2,lons_edgar[-1]+lon_res,lon_res)

grid_in   = {'lon':srclon_cen,'lat':srclat_cen, 'lon_b':srclon_cor,'lat_b':srclat_cor}
grid_out  = {'lon':wrf_lon[0],'lat':wrf_lat[0], 'lon_b':wrf_lon_c[0],'lat_b':wrf_lat_c[0]}

In [13]:
regridder = xe.Regridder(grid_in,grid_out,regrid_method,reuse_weights=False)
print(regridder)
edgar_data_re = {};
for v,pol in enumerate(var):
    edgar_data_re[str(pol)]  = regridder(edgar_data[str(pol)])  

xESMF Regridder 
Regridding algorithm:       conservative 
Weight filename:            conservative_1800x3600_124x204.nc 
Reuse pre-computed weights? False 
Input grid shape:           (1800, 3600) 
Output grid shape:          (124, 204) 
Periodic in longitude?      False


/home/nonna/anaconda3/envs/xesmf/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
/home/nonna/anaconda3/envs/xesmf/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


In [14]:
print('generating netcdf file..')
wrf_input     = wrf_geo
dim_emi_items = list(wrf_input.dimensions.items())
domain        = dom
projection    = projection_wrf

bytime        = pd.date_range("2022-08-01", freq="D", periods=31)
nhoras        = ["{:02d}".format(n) for n in np.arange(0,24,1)]

generating netcdf file..


In [16]:

#for q,m in enumerate(bytime):
#    for d in nhoras:
q = 10
m = bytime[10]
d = nhoras[10]
date          = pd.to_datetime(m).strftime('%Y-%m-%d')+ '_'+d+':00:00'
filename      = edgar_out + 'wrfchemi_d0%s_%s.nc'%(dom,str(date).replace(":","_"))
dataset       = cdf.Dataset(filename,'w',format='NETCDF4_CLASSIC')

# Dimensions
lon           = dataset.createDimension(dim_emi_items[2][0], wrf_input.getncattr('WEST-EAST_PATCH_END_UNSTAG'))
lat           = dataset.createDimension(dim_emi_items[3][0], wrf_input.getncattr('SOUTH-NORTH_PATCH_END_UNSTAG'))
levels        = dataset.createDimension('emissions_zdim_stag',10)
len           = dataset.createDimension('DateStrLen',19)
time          = dataset.createDimension('Time', 1)


# Variables
time          = dataset.createVariable('Times', 'S1',('Time','DateStrLen'))
lat           = dataset.createVariable('XLAT',  'f4', (dim_emi_items[3][0],dim_emi_items[2][0]))
lon           = dataset.createVariable('XLONG', 'f4', (dim_emi_items[3][0],dim_emi_items[2][0]))

co2f          = dataset.createVariable('E_CO2', 'f4', ('Time','emissions_zdim_stag',dim_emi_items[3][0],dim_emi_items[2][0]))
cof           = dataset.createVariable('E_CO',  'f4', ('Time','emissions_zdim_stag',dim_emi_items[3][0],dim_emi_items[2][0]))
#co2f          = dataset.createVariable('E_CO2', 'f4', ('Time','emissions_zdim',dim_emi_items[3][0],dim_emi_items[2][0]))
ch4f          = dataset.createVariable('E_CH4', 'f4', ('Time','emissions_zdim_stag',dim_emi_items[3][0],dim_emi_items[2][0]))
# data
time[:,:]     = np.array([" ".join(date).split()], dtype = 'S19')
lat[:,:]      = np.array(wrf_input.variables['XLAT_M'][0])
lon[:,:]      = np.array(wrf_input.variables['XLONG_M'][0])
ch4f[0,0,:,:] = edgar_data_re["CH4"]
co2f[0,0,:,:] = edgar_data_re["CO2"]
cof[0,0,:,:]  = edgar_data_re["CO"]
for q in range(1,10):
    #print(q)
    ch4f[0,q,:,:] = np.zeros(edgar_data_re["CH4"].shape)
    ch4f[0,q,:,:] = np.zeros(edgar_data_re["CH4"].shape)
    ch4f[0,q,:,:] = np.zeros(edgar_data_re["CH4"].shape)

# Attributes global
for i,j in enumerate(wrf_i.ncattrs()):
    if i == 0: continue
    dataset.setncattr(j,wrf_i.getncattr(j))

# Attributes variables
lon.MemoryOrder  = 'XY'
lon.description  = 'LONGITUDE, WEST IS NEGATIVE'
lon.untis        = 'degree east'
lon.long_name    = " "
lon.FieldType    = 104

lat.MemoryOrder  = 'XY'
lat.description  = 'LATITUDE, SOUTH IS NEGATIVE'
lat.units        = 'degree north'
lat.stagger      = ""
lat.FieldType    = 104

co2f.MemoryOrder  = 'XYZ'
co2f.description  = 'EMISSIONS'
co2f.units        = 'mol km^-2 hr^-1'
co2f.stagger       = "Z"
co2f.FieldType    = 104


ch4f.MemoryOrder  = 'XYZ'
ch4f.description  = 'EMISSIONS'
ch4f.units        = 'mol km^-2 hr^-1'
ch4f.stagger       = "Z"
ch4f.FieldType    = 104


cof.MemoryOrder  = 'XYZ'
cof.description  = 'EMISSIONS'
cof.units        = 'mol km^-2 hr^-1'
cof.stagger       = "Z"
cof.FieldType    = 104

#co2f.MemoryOrder  = 'XYZ'
#co2f.description  = 'EMISSIONS'
#co2f.units        = 'mol $km^-2 hr^⁻1$'
#co2f.stagger       = "Z"
#co2f.FieldType    = 104

dataset.close()

print('DONE WITH GHG fire emissions REGRID-PREPROCESSING')

DONE WITH GHG fire emissions REGRID-PREPROCESSING
